In [1]:
# Criar um navegador
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd

nav = webdriver.Chrome()
# Importar/visualizar a base de dados
tabela_produtos = pd.read_excel("buscas.xlsx")
display(tabela_produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64 gb,mini watch,2000,3500
1,rtx 3060,zota galax,3000,4500


In [2]:
import time 
# Pesquisar pelo produto
produto = "iphone 12 64 gb"
termos_banidos = "mini watch"
preco_min = 2000
preco_max = 4000

In [8]:
def verificar_tem_termos_banidos(lista_termos_banidos, nome):
    tem_termos_banidos = False
    for palavra in lista_termos_banidos:
        if palavra in nome:
            tem_termos_banidos = True
    return tem_termos_banidos

def verificar_tem_todos_termos_banidos(lista_termos_nome_produto, nome):
    tem_todos_termos_produtos = True
    for palavra in lista_termos_nome_produto:
        if palavra not in nome:
            tem_todos_termos_produtos = False
    return tem_todos_termos_produtos

def busca_google_shopping(nav, produto, termos_banidos, preco_min, preco_max):
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_nome_produto = produto.split(" ")
    lista_ofertas = []
    preco_min = float(preco_min)
    preco_max = float(preco_max)

    # Entrar no google
    nav.get("https://www.google.com/")
    nav.find_element('xpath', '//*[@id="APjFqb"]').send_keys(produto, Keys.ENTER)

    # Entrar na aba shopping
    time.sleep(2)
    elementos = nav.find_elements('class name', 'O3S9Rb')
    for item in elementos:
        if "Shopping" in item.text:
            item.click()
            break

    # Pegar as informações do produtos
    lista_resultados = nav.find_elements('class name', 'i0X6df')

    for resultado in lista_resultados:    
        nome = resultado.find_element('class name', 'tAxDx').text
        nome = nome.lower()

        # Analisar se ele não tem nenhum termo banido:
        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)
       
        # Analisar se ele tem TODOS os termos do nome do produto
        tem_todos_termos_produtos = verificar_tem_todos_termos_banidos(lista_termos_nome_produto, nome)
        
        # Selecionar só os elementos que tem_termos_banidos = False e ao mesmo tempo tem_todos_termos_produtos = False 
        if not tem_termos_banidos and tem_todos_termos_produtos:
            try:

                preco = resultado.find_element('class name', 'a8Pemb').text
                preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
                preco = float(preco)

                # se o preco esta entre preco_min e preco_max
                if preco_min <= preco <= preco_max:
                    elemento_referencia = resultado.find_element('class name', 'bONr3b')
                    elemento_pai = elemento_referencia.find_element('xpath', '..')
                    link = elemento_pai.get_attribute('href')

                    lista_ofertas.append((nome, preco, link))
            except:
                continue

    return lista_ofertas

In [6]:
def busca_buscape(nav, produto, termos_banidos, preco_min, preco_max):
    # Tratar valores:
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_nome_produto = produto.split(" ")
    lista_ofertas = []
    preco_min = float(preco_min)
    preco_max = float(preco_max)
    
    # Entrar no site
    nav.get("https://www.buscape.com.br/")
    nav.find_element('xpath', '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(produto, Keys.ENTER)
    
    # Pegar os resultados
    time.sleep(3)
    lista_resultados = nav.find_elements('class name', 'Paper_Paper__HIHv0')
       
    for resultado in lista_resultados:
        preco = resultado.find_element(By.CLASS_NAME, 'Text_MobileHeadingS__Zxam2').text
#         nome = resultado.get_attribute('title')
        nome = resultado.find_element(By.CLASS_NAME, 'SearchCard_ProductCard_NameWrapper__Gv0x_').text
        nome = nome.lower()
        link = resultado.get_attribute('href')
        
        # Analisar se ele não tem nenhum termo banido:
        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)
       
        # Analisar se ele tem TODOS os termos do nome do produto
        tem_todos_termos_produtos = verificar_tem_todos_termos_banidos(lista_termos_nome_produto, nome)
        
    # Analisar se o preço está entre o preço min e o preço maz
        if not tem_termos_banidos and tem_todos_termos_produtos:
            preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
            preco = float(preco)
            if preco_min <= preco <= preco_max:
                lista_ofertas.append((nome, preco, link))
                
    return lista_ofertas            

In [9]:
lista_ofertas_buscape = busca_buscape(nav, produto, termos_banidos, preco_min, preco_max)
print(lista_ofertas_buscape)

[('smartphone apple iphone 12 vermelho 64gb câmera dupla\n4.8 (38)', 3779.1, None), ('smartphone apple iphone 12 64gb câmera dupla\n4.8 (4619)', 3599.1, None), ('smartphone apple iphone 12 usado 64gb câmera dupla', 3109.0, None)]


In [11]:
lista_ofertas_google_shopping = busca_google_shopping(nav, produto, termos_banidos, preco_min, preco_max)

print(lista_ofertas_google_shopping)
                                                

[('usado: iphone 12 64gb preto muito bom - trocafone - apple', 2973.52, 'https://www.google.com/url?url=https://www.magazineluiza.com.br/usado-iphone-12-64gb-preto-muito-bom-trocafone-apple/p/ccg0461d77/te/ceba%3F%26seller_id%3Dtrocafone&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwjekumc0Pf_AhWdH7kGHWV5DAgQgOUECPYM&usg=AOvVaw0geDJgOOUUWfhbJqFvU085'), ('iphone 12 64gb preto de vitrine tela 6,1 4g câmera traseira 12mp 12mp vitrine', 3349.0, 'https://www.google.com/url?url=https://www.carrefour.com.br/iphone-12-64gb-preto-de-vitrine-tela-61-4g-camera-traseira-12mp-12mp-vitrine-mp932568692/p&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwjekumc0Pf_AhWdH7kGHWV5DAgQgOUECKoN&usg=AOvVaw2TNsdx4kdEv1UASxOjH5u-'), ('vitrine: iphone 12 apple 64gb azul tela 6,1 câmera traseira dupla 12mp', 3134.05, 'https://www.google.com/url?url=https://www.carrefour.com.br/vitrine-iphone-12-apple-64gb-azul-tela-61-camera-traseira-dupla-12mp-mp911874363/p&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwjekumc0Pf_AhWdH7kGHWV5DAgQguUECNMN&usg=AOvVaw2_LY4guTH

In [ ]:
# para cada item dentro da nossa base de dados, para cada produto
    # procurar esse produto no google shopping
        # verificar se algum dos produtos de google shopping está dentro da minha faixa de preço
    # procurar esse produto no buscapé
        # verificar se algum dos produtos de buscapé está dentro da minha faixa de preço
# salvar as ofertas boas em um dataframe (tabela)
# exportar pro excel
# enviar por email o resultado da tabela

In [13]:
tabela_ofertas = pd.DataFrame()

for linha in tabela_produtos.index:
    produto = tabela_produtos.loc[linha, "Nome"]
    termos_banidos = tabela_produtos.loc[linha, "Termos banidos"]
    preco_min = tabela_produtos.loc[linha, "Preço mínimo"]
    preco_max = tabela_produtos.loc[linha, "Preço máximo"]
    
    lista_ofertas_google_shopping = busca_google_shopping(nav, produto, termos_banidos, preco_min, preco_max)
    if lista_ofertas_google_shopping:
        tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns=['produto', 'preco', 'link'])
        tabela_ofertas = tabela_ofertas.append(tabela_google_shopping)
    else:
        tabela_google_shopping = None
        
    lista_ofertas_buscape = busca_buscape(nav, produto, termos_banidos, preco_min, preco_max)
    if lista_ofertas_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns=['produto', 'preco', 'link'])
        tabela_ofertas = tabela_ofertas.append(tabela_buscape)
    else:
        tabela_buscape = None

display(tabela_ofertas)

C:\Users\danil\AppData\Local\Temp\ipykernel_18752\3598176879.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela_ofertas = tabela_ofertas.append(tabela_google_shopping)
C:\Users\danil\AppData\Local\Temp\ipykernel_18752\3598176879.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela_ofertas = tabela_ofertas.append(tabela_buscape)
C:\Users\danil\AppData\Local\Temp\ipykernel_18752\3598176879.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela_ofertas = tabela_ofertas.append(tabela_google_shopping)
C:\Users\danil\AppData\Local\Temp\ipykernel_18752\3598176879.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela_

,produto,preco,link
0,usado: iphone 12 64gb preto muito bom - trocaf...,2973.52,https://www.google.com/url?url=https://www.mag...
1,"iphone 12 64gb preto de vitrine tela 6,1 4g câ...",3349.00,https://www.google.com/url?url=https://www.car...
2,"vitrine: iphone 12 apple 64gb azul tela 6,1 câ...",3134.05,https://www.google.com/url?url=https://www.car...
3,"iphone 12 64gb roxo tela 6,1 4g câmera traseir...",3349.00,https://www.google.com/url?url=https://www.car...
4,"vitrine: iphone 12 apple 64gb branco tela 6,1 ...",3134.05,https://www.google.com/url?url=https://www.car...
5,iphone 12 64gb - azul - estou zerado,2699.10,https://www.google.com/url?url=https://www.tro...
6,"vitrine: iphone 12 apple 64gb roxo tela 6,1 câ...",3134.05,https://www.google.com/url?url=https://www.car...
7,iphone 12 64gb - branco - estou zerado,2699.10,https://www.google.com/url?url=https://www.tro...
8,iphone 12 64gb vitrine swap + cabo e fonte - b...,2780.00,https://www.google.com/url?url=https://www.ata...
9,usado: iphone 12 64gb azul excelente - trocafo...,3343.12,https://www.google.com/url?url=https://www.mag...
